In [1]:
import mfem.ser as mfem
from glvis import glvis, to_stream

In [ ]:
glvis(data[, width=640, height=480])

# or assign if you want to update later
g = glvis(data)
# run a cell with `g` to show it
g

In [ ]:
import mfem.ser as mfem

# create sample mesh for square shape
mesh = mfem.Mesh(10, 10, "TRIANGLE")

# create finite element function space
fec = mfem.H1_FECollection(1, mesh.Dimension())   # H1 order=1
fespace = mfem.FiniteElementSpace(mesh, fec)

#
ess_tdof_list = mfem.intArray()
ess_bdr = mfem.intArray([1]*mesh.bdr_attributes.Size())
fespace.GetEssentialTrueDofs(ess_bdr, ess_tdof_list)

# constant coefficient (diffusion coefficient and RHS)
alpha = mfem.ConstantCoefficient(1.0)
rhs = mfem.ConstantCoefficient(1.0)

# Note:
#    Diffusion coefficient can be variable. To use numba-JIT compiled
#    functio. Use the following, where x is numpy-like array.
# @mfem.jit.scalar
# def alpha(x):
#     return x+1.0
#

# define Bilinear and Linear operator
a = mfem.BilinearForm(fespace)
a.AddDomainIntegrator(mfem.DiffusionIntegrator(alpha))
a.Assemble()
b = mfem.LinearForm(fespace)
b.AddDomainIntegrator(mfem.DomainLFIntegrator(rhs))
b.Assemble()

# create gridfunction, which is where the solution vector is stored
x = mfem.GridFunction(fespace);
x.Assign(0.0)

# form linear equation (AX=B)
A = mfem.OperatorPtr()
B = mfem.Vector()
X = mfem.Vector()
a.FormLinearSystem(ess_tdof_list, x, b, A, X, B);
print("Size of linear system: " + str(A.Height()))

# solve it using PCG solver and store the solution to x
AA = mfem.OperatorHandle2SparseMatrix(A)
M = mfem.GSSmoother(AA)
mfem.PCG(AA, M, B, X, 1, 200, 1e-12, 0.0)
a.RecoverFEMSolution(X, b, x)

# extract vertices and solution as numpy array
verts = mesh.GetVertexArray()
sol = x.GetDataArray()

# plot solution using Matplotlib

import matplotlib.pyplot as plt
import matplotlib.tri as tri

triang = tri.Triangulation(verts[:,0], verts[:,1])

fig1, ax1 = plt.subplots()
ax1.set_aspect('equal')
tpc = ax1.tripcolor(triang, sol, shading='gouraud')
fig1.colorbar(tpc)
plt.show()

Size of linear system: 121
   Iteration :   0  (B r, r) = 0.00663539
   Iteration :   1  (B r, r) = 0.00371547
   Iteration :   2  (B r, r) = 0.000455545
   Iteration :   3  (B r, r) = 1.36394e-05
   Iteration :   4  (B r, r) = 4.13047e-07
   Iteration :   5  (B r, r) = 2.24933e-08
   Iteration :   6  (B r, r) = 6.42285e-10
   Iteration :   7  (B r, r) = 3.89967e-11
   Iteration :   8  (B r, r) = 5.72023e-12
   Iteration :   9  (B r, r) = 1.02049e-13
   Iteration :  10  (B r, r) = 3.54358e-16
Average reduction factor = 0.21696


TypeError: tuple indices must be integers or slices, not tuple

In [15]:
import mfem.ser as mfem
from glvis import glvis, to_stream

In [16]:
# Load the mesh from a local file
# meshfile = '../../mfem/data/star.mesh'
# mesh = mfem.Mesh(meshfile)

# Alternatively, create a simple square mesh and refine it
mesh = mfem.Mesh(5, 5, "TRIANGLE")
mesh.UniformRefinement()

# Create H1 finite element function space
fec = mfem.H1_FECollection(2, mesh.Dimension()) # order=2
fespace = mfem.FiniteElementSpace(mesh, fec)      

# Determine essential degrees of freedom (the whole boundary here)
ess_tdof_list = mfem.intArray()
ess_bdr = mfem.intArray([1]*mesh.bdr_attributes.Size())
fespace.GetEssentialTrueDofs(ess_bdr, ess_tdof_list)

# Define Bilinear and Linear forms for the Laplace problem -Δu=1
one = mfem.ConstantCoefficient(1.0)
a = mfem.BilinearForm(fespace)
a.AddDomainIntegrator(mfem.DiffusionIntegrator(one))
a.Assemble()
b = mfem.LinearForm(fespace)
b.AddDomainIntegrator(mfem.DomainLFIntegrator(one))
b.Assemble()

# Create a grid function for the solution and initialize with 0
x = mfem.GridFunction(fespace);
x.Assign(0.0)

# Form the linear system, AX=B, for the FEM discretization
A = mfem.OperatorPtr()
B = mfem.Vector()
X = mfem.Vector()
a.FormLinearSystem(ess_tdof_list, x, b, A, X, B);
print("Size of the linear system: " + str(A.Height()))

# Solve the system using PCG solver and get the solution in x
Asm = mfem.OperatorHandle2SparseMatrix(A)
Msm = mfem.GSSmoother(Asm)
mfem.PCG(Asm, Msm, B, X, 1, 200, 1e-12, 0.0)
a.RecoverFEMSolution(X, b, x)

Size of the linear system: 441
   Iteration :   0  (B r, r) = 0.00127274
   Iteration :   1  (B r, r) = 0.00349488
   Iteration :   2  (B r, r) = 0.00177954
   Iteration :   3  (B r, r) = 0.00127536
   Iteration :   4  (B r, r) = 0.000715054
   Iteration :   5  (B r, r) = 0.000259268
   Iteration :   6  (B r, r) = 9.09953e-05
   Iteration :   7  (B r, r) = 9.86878e-06
   Iteration :   8  (B r, r) = 1.00827e-06
   Iteration :   9  (B r, r) = 3.5336e-07
   Iteration :  10  (B r, r) = 2.78374e-08
   Iteration :  11  (B r, r) = 5.71184e-09
   Iteration :  12  (B r, r) = 4.61089e-10
   Iteration :  13  (B r, r) = 7.13477e-11
   Iteration :  14  (B r, r) = 5.73371e-11
   Iteration :  15  (B r, r) = 1.75715e-11
   Iteration :  16  (B r, r) = 8.13841e-13
   Iteration :  17  (B r, r) = 1.67974e-13
   Iteration :  18  (B r, r) = 3.59872e-14
   Iteration :  19  (B r, r) = 1.89648e-14
   Iteration :  20  (B r, r) = 2.07061e-14
   Iteration :  21  (B r, r) = 3.14143e-15
   Iteration :  22  (B r, r)

In [17]:
# Plot the mesh + solution (all GLVis keys and mouse commands work)
glvis((mesh, x), 400, 400)

glvis(data_str='MFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see mesh/geom.hpp):\n#\n# POINT       = 0\n# SEGME…

In [6]:
# Plot the mesh only
glvis(mesh)

glvis(data_str='MFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see mesh/geom.hpp):\n#\n# POINT       = 0\n# SEGME…

In [7]:
g = glvis(to_stream(mesh,x) + 'keys ARjlmcbp*******')
g.set_size(600, 400)
g

glvis(data_str='MFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see mesh/geom.hpp):\n#\n# POINT       = 0\n# SEGME…

In [12]:
import pyglvis
import numpy as np

# Crea algunos datos de ejemplo
x = np.linspace(-1, 1, 100)
y = np.linspace(-1, 1, 100)
X, Y = np.meshgrid(x, y)
Z = np.sin(X**2 + Y**2)

# Crea una visualización con PyGLVis
viewer = pyglvis.viewer(show_boundary=False)
viewer.add_surface(x, y, Z)

# Ajusta la vista según sea necesario
viewer.set_view(azimuth=30, elevation=30, distance=2)

# Exporta la visualización como un archivo .jpg
viewer.save_image("output.jpg", width=800, height=600, quality=95)


AttributeError: module 'pyglvis' has no attribute 'viewer'